# Introduction

In this notebook, I will load a flatfile download of the reddit whisky reviews, parse the file, and use the URL field to query Reddit and acquire reviews.

In [10]:
import os
import uuid
import praw
import pandas as pd
import pickle as pkl
import time

In [3]:
def url_fixer(url):
    '''
    Function to test and fix URLs.  If the URL contains the reddit /r/ we will split there and replace with
    'https://www.reddit.com/r/'.  Otherwise, we will ensure the URL starts with http[s]:// and work from there. 
    '''
    
    if '/r/' in url:
        # There are a couple URLs where the URL is pasted twice.  Splitting on /r/ and getting the
        # last element deals with cases where there are too many values to unpack due to this double-pasting
        sub = url.split('/r/')[-1]
        return 'https://www.reddit.com/r/' + sub
    else:
        if url.startswith('http'):
            if ('reddit.com' in url) | ('redd.it' in url):
                return url
        else:
            print('url error:', url) 
            return None
        
def get_reddit_sub(url): 
    '''
    Helper Function t get reddit submissions for API - build in try/except for lambda function
    '''
    try:
        return reddit.submission(url=url)
        time.sleep(1)
    except:
        return None

## Load the Data File

In [4]:
whisky_data = pd.read_csv('../data/Reddit_Whisky_Network_Review_Archive_20190424.csv')

In [5]:
whisky_data.head()

,Timestamp,Whisky Name,Reviewer's Reddit Username,Link To Reddit Review,Reviewer Rating,Whisky Region or Style,Full Bottle Price Paid,Date of Review
0,12/14/2012 10:03:18,100 Pipers,merlinblack,http://www.reddit.com/r/Scotch/comments/14uder...,68,Blend,NaN,12/14/12
1,11/4/2018 0:50:39,1792 225th Anniversary,scottmotorrad,https://www.reddit.com/r/bourbon/comments/9ssq...,8,Bourbon,NaN,10/30/18
2,11/17/2017 8:15:22,1792 225th Anniversary,WildOscar66,https://www.reddit.com/r/bourbon/comments/7445...,80,Bourbon,NaN,10/17/17
3,11/4/2018 0:49:15,1792 Bottled In Bond,scottmotorrad,https://www.reddit.com/r/bourbon/comments/9ryi...,80,Bourbon,$34.99,10/27/18
4,12/19/2018 0:38:02,1792 Bottled In Bond Oak Liquor Cabinet Pick,scottmotorrad,https://www.reddit.com/r/bourbon/comments/a7jw...,90,Bourbon,NaN,12/19/2018


Let's give each row it's own unique_id we can refer back to in databases and such.

In [5]:
whisky_data['uuid'] = whisky_data.Timestamp.apply(lambda x: uuid.uuid4().hex)

In [7]:
#whisky_data.to_csv('../data/Reddit_Whisky_Network_Review_Archive_20190424_wuuid.csv')
whisky_data = pd.read_csv('../data/Reddit_Whisky_Network_Review_Archive_20190424_wuuid.csv')

In [8]:
whisky_data.head()

,Unnamed: 0,Timestamp,Whisky Name,Reviewer's Reddit Username,Link To Reddit Review,Reviewer Rating,Whisky Region or Style,Full Bottle Price Paid,Date of Review,uuid
0,0,12/14/2012 10:03:18,100 Pipers,merlinblack,http://www.reddit.com/r/Scotch/comments/14uder...,68,Blend,NaN,12/14/12,1497ef54838b47d7a34f4085720a33da
1,1,11/4/2018 0:50:39,1792 225th Anniversary,scottmotorrad,https://www.reddit.com/r/bourbon/comments/9ssq...,8,Bourbon,NaN,10/30/18,5bfcb7bf5f454473b2042da21a0055a8
2,2,11/17/2017 8:15:22,1792 225th Anniversary,WildOscar66,https://www.reddit.com/r/bourbon/comments/7445...,80,Bourbon,NaN,10/17/17,29ec172373944280ad2c2d0158db2928
3,3,11/4/2018 0:49:15,1792 Bottled In Bond,scottmotorrad,https://www.reddit.com/r/bourbon/comments/9ryi...,80,Bourbon,$34.99,10/27/18,bffcc71c3ad64363a899fcac1be0cb46
4,4,12/19/2018 0:38:02,1792 Bottled In Bond Oak Liquor Cabinet Pick,scottmotorrad,https://www.reddit.com/r/bourbon/comments/a7jw...,90,Bourbon,NaN,12/19/2018,2db4d5ef774240a381d127cb4b41d2df


## URL Check

Let's double-check that URLs are properly formatted, and at least have a /r/ substring in them that we can work from to build up proper URLS. 

In [8]:
whisky_data['url_test'] = whisky_data['Link To Reddit Review'].apply(lambda x: '/r/' in x or 'reddit.com' in x or 'redd.it' in x)

In [11]:
whisky_data.url_test.value_counts()

url_test
True     31194
False       36
Name: count, dtype: int64

There are 36 observations where we don't have a reddit /r/ in the URL or appear to correspond to a reddit post. Let's take a look at those.

In [12]:
whisky_data[whisky_data.url_test == False]

,Timestamp,Whisky Name,Reviewer's Reddit Username,Link To Reddit Review,Reviewer Rating,Whisky Region or Style,Full Bottle Price Paid,Date of Review,uuid,url_test
275,4/23/2016 17:34:00,Aberlour 12 Non Chill Filtered,bieliebielie,https://cdn2.masterofmalt.com/whiskies/p-2813/...,88,Speyside,40,04/23/16,191719c5d44e4d448f72d1c94cf23e1b,False
610,5/21/2016 16:59:06,Aberlour A'bunadh Batch #53,Shawshankreddit,https://i.reddituploads.com/bdcde4da945247c9b0...,91,Speyside,£40,04/06/16,1f58b80c90114feea352b8d8999160c4,False
2077,5/27/2016 20:40:42,Ardbeg Uigeadail,saltpeanutss,https://c2.staticflickr.com/8/7059/6905284987_...,91,Islay,62,05/27/16,e1e69eea55204ae59038436e41ad4e71,False
2604,1/13/2015 14:48:22,Arran Orkney Bere,gibbonwl,http://i.imgur.com/T20WB2F.jpg,95,Island,73,01/13/15,fdfcbac39e4345cd95d5d3e6ba66a925,False
3361,12/18/2014 12:12:47,Balvenie 12 Doublewood,CandyMan77,http://i.imgur.com/0lujJqz.jpg,86,Speyside,90,12/18/14,05d1bb72a4ea45929fa198204c969c60,False
4964,7/10/2018 13:48:51,Bernheim Original Small Batch Wheat Whiskey,FuzzyWildcat,74,74,Wheat,NaN,1/6/18,a3c4881233dd4ec8ad38a55f55ae784d,False
7095,1/10/2015 15:57:09,Bruichladdich Scottish Barley The Classic Laddie,gildedrain,http://i.imgur.com/05FxUjj.jpg,89,Islay,57,01/10/15,206365f262f94913bbf753acc9c97654,False
7184,4/27/2016 22:03:02,Bruichladdich The Laddie Ten,lookitskeith,https://dokla.net/dailydram/wp-content/uploads...,83,Islay,NaN,04/27/16,bd0bffa175734e26a7a65a82fbbcd007,False
8837,10/18/2016 13:31:26,Cardhu 12,bpnelson7,http://imgur.com/a/JB941,85,Speyside,43,10/18/2016,f580e6cdb2d64a42b94b057c0639b2e7,False
11627,8/4/2015 20:11:50,Evan Williams Single Barrel,Lasidar,http://whiskeyapostle.com/wp-content/uploads/2...,83,Bourbon,NaN,08/04/15,c919f7264d5a4f02ac9e1f9bf313db58,False


Of the 36, most correspond to URLs outside the Reddit site, where we can't scrape with the Reddit API.  For now, we will ignore these reviews, since they only account for 0.1% of reviews.

### Fix Invalid URLs that don't start with http

In [13]:
whisky_data['parsed_url'] = whisky_data['Link To Reddit Review'].apply(url_fixer)

url error: 74
url error: 78
url error: spg2469
url error: Link To Reddit Review


In [14]:
whisky_data.head()

,Timestamp,Whisky Name,Reviewer's Reddit Username,Link To Reddit Review,Reviewer Rating,Whisky Region or Style,Full Bottle Price Paid,Date of Review,uuid,url_test,parsed_url
0,12/14/2012 10:03:18,100 Pipers,merlinblack,http://www.reddit.com/r/Scotch/comments/14uder...,68,Blend,NaN,12/14/12,1497ef54838b47d7a34f4085720a33da,True,https://www.reddit.com/r/Scotch/comments/14ude...
1,11/4/2018 0:50:39,1792 225th Anniversary,scottmotorrad,https://www.reddit.com/r/bourbon/comments/9ssq...,8,Bourbon,NaN,10/30/18,5bfcb7bf5f454473b2042da21a0055a8,True,https://www.reddit.com/r/bourbon/comments/9ssq...
2,11/17/2017 8:15:22,1792 225th Anniversary,WildOscar66,https://www.reddit.com/r/bourbon/comments/7445...,80,Bourbon,NaN,10/17/17,29ec172373944280ad2c2d0158db2928,True,https://www.reddit.com/r/bourbon/comments/7445...
3,11/4/2018 0:49:15,1792 Bottled In Bond,scottmotorrad,https://www.reddit.com/r/bourbon/comments/9ryi...,80,Bourbon,$34.99,10/27/18,bffcc71c3ad64363a899fcac1be0cb46,True,https://www.reddit.com/r/bourbon/comments/9ryi...
4,12/19/2018 0:38:02,1792 Bottled In Bond Oak Liquor Cabinet Pick,scottmotorrad,https://www.reddit.com/r/bourbon/comments/a7jw...,90,Bourbon,NaN,12/19/2018,2db4d5ef774240a381d127cb4b41d2df,True,https://www.reddit.com/r/bourbon/comments/a7jw...


## Pull URLs and Query with PRAW

In [15]:
reddit = praw.Reddit(client_id=os.getenv('reddit_clientid'),
                     client_secret=os.getenv('reddit_secretid'),
                     user_agent='WhiskeyReviewsScraper:0.1 by u/jbeck22')

In [16]:
whisky_queries = whisky_data[whisky_data.url_test == True][['uuid','parsed_url']]

In [17]:
whisky_queries.shape

(31194, 2)

Let's only look at the first 1000 to see how long things take.

In [18]:
test_urls = whisky_queries['parsed_url'].tolist()[0:50]

In [19]:
posts = [reddit.submission(url=x) for x in test_urls]

So the objects here are still lazy, and we'll need to actually pull the post contents for each one. What happens if we just pull the submission titles?

In [20]:
for i, post in enumerate(posts):
    try:
        title = post.title
        time.sleep(0.75)
    except Exception as e:
        print(e,'iteration ',i)

    if i%10 == 0:
        print(f"Iteration {i} of {len(posts)}")

Iteration 1 of 50
Iteration 2 of 50
Iteration 3 of 50
Iteration 4 of 50
Iteration 5 of 50
Iteration 6 of 50
Iteration 7 of 50
Iteration 8 of 50
Iteration 9 of 50
Iteration 11 of 50
Iteration 12 of 50
Iteration 13 of 50
Iteration 14 of 50
Iteration 15 of 50
Iteration 16 of 50
Iteration 17 of 50
Iteration 18 of 50
Iteration 19 of 50
Iteration 21 of 50
Iteration 22 of 50
Iteration 23 of 50
Iteration 24 of 50
Iteration 25 of 50
Iteration 26 of 50
Iteration 27 of 50
Iteration 28 of 50
Iteration 29 of 50
Iteration 31 of 50
Iteration 32 of 50
Iteration 33 of 50
Iteration 34 of 50
Iteration 35 of 50
Iteration 36 of 50
Iteration 37 of 50
Iteration 38 of 50
Iteration 39 of 50
Iteration 41 of 50
Iteration 42 of 50
Iteration 43 of 50
Iteration 44 of 50
Iteration 45 of 50
Iteration 46 of 50
Iteration 47 of 50
Iteration 48 of 50
Iteration 49 of 50


In [21]:
vars(posts[0])

{'comment_limit': 2048,
 'comment_sort': 'confidence',
 'id': '14uder',
 '_reddit': <praw.reddit.Reddit at 0x11429bdd0>,
 '_fetched': True,
 '_additional_fetch_params': {},
 '_comments_by_id': {'t1_c7ghjy2': Comment(id='c7ghjy2'),
  't1_c7grqm4': Comment(id='c7grqm4'),
  't1_c7grt6y': Comment(id='c7grt6y'),
  't1_c7gsa0x': Comment(id='c7gsa0x'),
  't1_c7gsale': Comment(id='c7gsale'),
  't1_c7vuiso': Comment(id='c7vuiso'),
  't1_c7wwrpc': Comment(id='c7wwrpc'),
  't1_c7wygrn': Comment(id='c7wygrn'),
  't1_c7gyxy8': Comment(id='c7gyxy8'),
  't1_c7h88ms': Comment(id='c7h88ms'),
  't1_c7gvn73': Comment(id='c7gvn73'),
  't1_c7h89g3': Comment(id='c7h89g3'),
  't1_c7hc4h0': Comment(id='c7hc4h0'),
  't1_c7ghkiw': Comment(id='c7ghkiw'),
  't1_c7gipg6': Comment(id='c7gipg6'),
  't1_c7j5mlg': Comment(id='c7j5mlg'),
  't1_c7gq4x6': Comment(id='c7gq4x6'),
  't1_c7grpk5': Comment(id='c7grpk5'),
  't1_c7hvru3': Comment(id='c7hvru3'),
  't1_c7gr9dt': Comment(id='c7gr9dt'),
  't1_c7grp1s': Comment(id='

In [20]:
posts[0].selftext

''

Some of these posts are experiencing 403 Errors and not returning anything.  It isn't clear what's happening though

In [24]:
test_urls[663]

'https://www.reddit.com/r/ScotchSwap/comments/59ioer/its_happening_2016_mystery_swap_review_thread_go/da1xgae/'

In [ ]:
test_urls[721]

In [ ]:
test_urls[851]

These are all coming from /r/ScotchSwap, which has been _BANNED BY REDDIT_

Either way, pulling titles results in the submission being collected, so it seems like that's all we'll need to do to save off the text

## Let's do the thing

In [22]:
whisky_queries

,uuid,parsed_url
0,1497ef54838b47d7a34f4085720a33da,https://www.reddit.com/r/Scotch/comments/14ude...
1,5bfcb7bf5f454473b2042da21a0055a8,https://www.reddit.com/r/bourbon/comments/9ssq...
2,29ec172373944280ad2c2d0158db2928,https://www.reddit.com/r/bourbon/comments/7445...
3,bffcc71c3ad64363a899fcac1be0cb46,https://www.reddit.com/r/bourbon/comments/9ryi...
4,2db4d5ef774240a381d127cb4b41d2df,https://www.reddit.com/r/bourbon/comments/a7jw...
...,...,...
31225,36c8c8e3ca1d497fad4eeb53a4db6173,https://www.reddit.com/r/Scotch/comments/beti9...
31226,7d48731dc1bc4c56a39f8ff4d7612a53,https://www.reddit.com/r/Scotch/comments/bfouc...
31227,3005814542914bbf8895a1d8a597d530,https://www.reddit.com/r/bourbon/comments/bgaf...
31228,52bf3e6154f4464685a8a15117a5ad4f,https://www.reddit.com/r/Scotch/comments/bgoqk...


In [23]:
whisky_queries['submission_obj'] = whisky_queries.parsed_url.apply(get_reddit_sub)

In [15]:
def is_fetched(submission):
    if submission is None:
        return False
    else:
        return submission._fetched
    
    
def first_comment(submission):
    try:
        submission.comment_sort = 'old'
        return list(submission.comments)[0].body
    except:
        return None
    
def selftext(submission):
    try:
        return submission.selftext
    except:
        return None

In [30]:
whisky_queries['no_sub'] = whisky_queries.submission_obj.isnull()

In [31]:
whisky_queries['fetched_flag'] = whisky_queries.submission_obj.apply(is_fetched)

In [32]:
whisky_queries.no_sub.value_counts()

no_sub
False    31186
True         8
Name: count, dtype: int64

In [33]:
whisky_queries.fetched_flag.value_counts()

fetched_flag
False    31094
True       100
Name: count, dtype: int64

In [59]:
new_i = 2500

while new_i < whisky_queries.shape[0]:
    print(f"Starting Iteration {new_i}")
    for i, submission in enumerate(whisky_queries.submission_obj.tolist()[new_i:new_i+500]):

        if submission is None or submission._fetched == True:
            continue

        try:
            title = submission.title
            time.sleep(0.75)
        except Exception as e:
            print(e, 'error on iteration ', i)

        if i%100 == 0:
            #print('Iteration Completed', i)
            whisky_queries.to_pickle('../data/whisky_reviews_intprogress.pkl')

    new_i+=500


Starting Iteration 2500
received 403 HTTP response error on iteration  72
received 403 HTTP response error on iteration  99
Starting Iteration 3000
received 403 HTTP response error on iteration  231
received 403 HTTP response error on iteration  235
received 403 HTTP response error on iteration  428
received 403 HTTP response error on iteration  480
Starting Iteration 3500
received 403 HTTP response error on iteration  27
received 403 HTTP response error on iteration  120
received 403 HTTP response error on iteration  405
Starting Iteration 4000
Starting Iteration 4500
Starting Iteration 5000
received 403 HTTP response error on iteration  35
received 403 HTTP response error on iteration  158
received 403 HTTP response error on iteration  456
Starting Iteration 5500
received 403 HTTP response error on iteration  26
received 403 HTTP response error on iteration  61
received 403 HTTP response error on iteration  133
received 403 HTTP response error on iteration  170
received 403 HTTP resp

In [60]:
whisky_queries.to_pickle('../data/whisky_reviews_intprogress.pkl')

In [11]:
whisky_queries = pd.read_pickle('../data/whisky_reviews_intprogress.pkl')

In [12]:
len(whisky_queries)

31194

In [13]:
vars(whisky_queries['submission_obj'].tolist()[500])

{'comment_limit': 2048,
 'comment_sort': 'confidence',
 'id': '32plw2',
 '_reddit': <praw.reddit.Reddit at 0x1103ad2e0>,
 '_fetched': True,
 '_additional_fetch_params': {},
 '_comments_by_id': {'t1_cqddx1t': Comment(id='cqddx1t'),
  't1_cqddxoo': Comment(id='cqddxoo')},
 'approved_at_utc': None,
 'subreddit': Subreddit(display_name='Scotch'),
 'selftext': '',
 'user_reports': [],
 'saved': False,
 'mod_reason_title': None,
 'gilded': 0,
 'clicked': False,
 'title': 'CS Sherry Reviews #20-25',
 'link_flair_richtext': [],
 'subreddit_name_prefixed': 'r/Scotch',
 'hidden': False,
 'pwls': 6,
 'link_flair_css_class': 'review',
 'downs': 0,
 'thumbnail_height': 78,
 'top_awarded_type': None,
 'hide_score': False,
 'name': 't3_32plw2',
 'quarantine': False,
 'link_flair_text_color': None,
 'upvote_ratio': 0.98,
 'author_flair_background_color': '',
 'subreddit_type': 'public',
 'ups': 51,
 'total_awards_received': 0,
 'media_embed': {},
 'thumbnail_width': 140,
 'author_flair_template_id': N

In [16]:
whisky_queries['first_comment'] = whisky_queries['submission_obj'].apply(first_comment)

/var/folders/t1/r7nzywq96z7_1663xk78kpmc0000gn/T/ipykernel_55719/2428232927.py:10: UserWarning: The comments for this submission have already been fetched, so the updated comment_sort will not have any effect.
  submission.comment_sort = 'old'


In [18]:
whisky_queries['selftext'] = whisky_queries['submission_obj'].apply(selftext)

In [19]:
whisky_queries['fetched_flag'] = whisky_queries.submission_obj.apply(is_fetched)

In [25]:
whisky_queries.uuid.value_counts()

uuid
c0e4165324544715aee2d112a070c34f    1
1497ef54838b47d7a34f4085720a33da    1
5bfcb7bf5f454473b2042da21a0055a8    1
29ec172373944280ad2c2d0158db2928    1
bffcc71c3ad64363a899fcac1be0cb46    1
                                   ..
501fd5bb9a5145ebb7afc719b954450f    1
554439749a3642f69ee758469aa741ad    1
d49129f74ced42c9a1163ee3d260d9f3    1
fa30f21b1e4b4deebf4a84d1474164a1    1
6c430247c0b243f4b3dd26dcd3c5ab0f    1
Name: count, Length: 31194, dtype: int64

In [26]:
review_dict = {}

for i, row in whisky_queries.iterrows():
    if row.fetched_flag == True:
        # get extra data
        uuid = row.uuid

        extra_data = whisky_data[whisky_data.uuid == uuid]

        review_dict[row.uuid] = {'submission': row.submission_obj,
                               'submission_id': row.submission_obj.id,
                               'whisky_name': extra_data['Whisky Name'].iloc[0],
                               'redditor_name': extra_data['Reviewer\'s Reddit Username'].iloc[0],
                               'region_or_style': extra_data['Whisky Region or Style'].iloc[0],
                               'rating': extra_data['Reviewer Rating'].iloc[0],
                               'review_date': extra_data['Date of Review'].iloc[0],
                               'first_comment': row.first_comment,
                               'selftext': row.selftext,
                               'title': row.submission_obj.title,
                               'subreddit': row.submission_obj.subreddit_name_prefixed,
                               'permalink': row.submission_obj.permalink,
                               'created_utc': row.submission_obj.created_utc
                            }

In [27]:
pkl.dump(review_dict, open('../data/review_dict.pkl', 'wb'), pkl.HIGHEST_PROTOCOL)

In [53]:
test = pkl.load(open('/mnt/data/processed/review_dict.pkl', 'rb'))

In [69]:
whisky_queries

,uuid,parsed_url,submission_obj,no_sub,fetched_flag,selftext,first_comment
0,1497ef54838b47d7a34f4085720a33da,https://www.reddit.com/r/Scotch/comments/14ude...,14uder,False,True,,My wife and I are on a trip to Thailand to mee...
1,5bfcb7bf5f454473b2042da21a0055a8,https://www.reddit.com/r/bourbon/comments/9ssq...,9ssqyk,False,True,[deleted],The 1792 series continues and this time we are...
2,29ec172373944280ad2c2d0158db2928,https://www.reddit.com/r/bourbon/comments/7445...,7445vx,False,True,,Bourbon Review #48 1792 225th Anniversary \n\n...
3,bffcc71c3ad64363a899fcac1be0cb46,https://www.reddit.com/r/bourbon/comments/9ryi...,9ryi8f,False,True,[removed],The 1792 review series continues! The Bottled ...
4,2db4d5ef774240a381d127cb4b41d2df,https://www.reddit.com/r/bourbon/comments/a7jw...,a7jwdd,False,True,[deleted],This week Oak Liquor Cabinet got in the long a...
...,...,...,...,...,...,...,...
31225,36c8c8e3ca1d497fad4eeb53a4db6173,https://www.reddit.com/r/Scotch/comments/beti9...,beti9a,False,False,,Linkwood 18y c1997 Exclusive Malts\n\nThis is ...
31226,7d48731dc1bc4c56a39f8ff4d7612a53,https://www.reddit.com/r/Scotch/comments/bfouc...,bfoucg,False,False,,Cadenhead Campbeltown Living Cask 29/12/17\n\n...
31227,3005814542914bbf8895a1d8a597d530,https://www.reddit.com/r/bourbon/comments/bgaf...,bgaf95,False,False,,/u/bananavanman MYSTERY 1.1 “BANFF”\n\nFrom a ...
31228,52bf3e6154f4464685a8a15117a5ad4f,https://www.reddit.com/r/Scotch/comments/bgoqk...,bgoqkp,False,False,,Bruichladdich Elements of Islay Br7 \n\nThis u...


In [70]:
whisky_data.head()

,Timestamp,Whisky Name,Reviewer's Reddit Username,Link To Reddit Review,Reviewer Rating,Whisky Region or Style,Full Bottle Price Paid,Date of Review,uuid,url_test,parsed_url
0,12/14/2012 10:03:18,100 Pipers,merlinblack,http://www.reddit.com/r/Scotch/comments/14uder...,68,Blend,NaN,12/14/12,1497ef54838b47d7a34f4085720a33da,True,https://www.reddit.com/r/Scotch/comments/14ude...
1,11/4/2018 0:50:39,1792 225th Anniversary,scottmotorrad,https://www.reddit.com/r/bourbon/comments/9ssq...,8,Bourbon,NaN,10/30/18,5bfcb7bf5f454473b2042da21a0055a8,True,https://www.reddit.com/r/bourbon/comments/9ssq...
2,11/17/2017 8:15:22,1792 225th Anniversary,WildOscar66,https://www.reddit.com/r/bourbon/comments/7445...,80,Bourbon,NaN,10/17/17,29ec172373944280ad2c2d0158db2928,True,https://www.reddit.com/r/bourbon/comments/7445...
3,11/4/2018 0:49:15,1792 Bottled In Bond,scottmotorrad,https://www.reddit.com/r/bourbon/comments/9ryi...,80,Bourbon,$34.99,10/27/18,bffcc71c3ad64363a899fcac1be0cb46,True,https://www.reddit.com/r/bourbon/comments/9ryi...
4,12/19/2018 0:38:02,1792 Bottled In Bond Oak Liquor Cabinet Pick,scottmotorrad,https://www.reddit.com/r/bourbon/comments/a7jw...,90,Bourbon,NaN,12/19/2018,2db4d5ef774240a381d127cb4b41d2df,True,https://www.reddit.com/r/bourbon/comments/a7jw...
